<a href="https://colab.research.google.com/github/Naomy-Yailin/SIS420/blob/main/laboratorio3/reg_log_onevsall_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este conjunto de datos consta de 101 animales de un zoológico.
# Hay 16 variables con varios rasgos para describir a los animales. **texto en negrita**
# Los 7 tipos de clase son: Mamífero, Ave, Reptil, Pez, Anfibio, Insecto e Invertebrado **texto en negrita**

# El propósito de este conjunto de datos es poder predecir la clasificación de los animales, en función de las variables. **texto en negrita**

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
import pandas as pd
from sklearn.model_selection import train_test_split


# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [29]:
# Cargar dataset Zoo
data = pd.read_csv('/content/zoo.csv')

# Extraer X (características) y y (etiquetas)
X = data.iloc[:, 1:-1].values   # todas las columnas menos 'animal_name' y 'class_type'
y = data.iloc[:, -1].values     # columna class_type

# Definir número de entradas y salidas
# Hay 16 atributos en el dataset, pero el comentario original contaba x0 (bias)
input_layer_size = X.shape[1] + 1   # 16 + 1 = 17
num_labels = len(np.unique(y))      # 7 clases distintas

# Ajustar etiquetas: pasar de 1-7 a 0-6 para evitar problemas
y = y - 1

# Número de ejemplos de entrenamiento
m = y.size

print("Número de ejemplos:", m)
print("Dimensión de X:", X.shape)
print("Clases distintas:", num_labels)
print("Primeras etiquetas:", y[:10])

Número de ejemplos: 101
Dimensión de X: (101, 16)
Clases distintas: 7
Primeras etiquetas: [0 0 3 0 0 0 0 3 3 0]


In [30]:
print(X[0,:])
print(y)

[1 0 0 1 0 0 1 1 1 1 0 0 4 0 0 1]
[0 0 3 0 0 0 0 3 3 0 0 1 3 6 6 6 1 0 3 0 1 1 0 1 5 4 4 0 0 0 5 0 0 1 3 0 0
 1 3 5 5 1 5 1 0 0 6 0 0 0 0 5 4 6 0 0 1 1 1 1 3 3 2 0 0 0 0 0 0 0 0 1 6 3
 0 0 2 6 1 1 2 6 3 1 0 6 3 1 5 4 2 2 3 0 0 1 0 5 0 6 1]


In [31]:
def featureNormalize(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0, ddof=1)  # ddof=1 para desviación estándar muestral
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

# Aplicar a tu dataset zoo
X_norm, mu, sigma = featureNormalize(X)

print("Primer ejemplo normalizado:", X_norm[0,:])
print("Medias:", mu)
print("Desviaciones estándar:", sigma)

Primer ejemplo normalizado: [ 1.15563073 -0.49443795 -1.17934447  1.20371316 -0.55551984 -0.74051504
  0.89197269  0.80575756  0.46337918  0.50980485 -0.29183867 -0.4476351
  0.56969831 -1.68998664 -0.38244559  1.13253179]
Medias: [0.42574257 0.1980198  0.58415842 0.40594059 0.23762376 0.35643564
 0.55445545 0.6039604  0.82178218 0.79207921 0.07920792 0.16831683
 2.84158416 0.74257426 0.12871287 0.43564356]
Desviaciones estándar: [0.49692121 0.40049474 0.49532468 0.4935224  0.42775027 0.48133478
 0.49950471 0.49151211 0.38460472 0.40784388 0.27140996 0.37601348
 2.03338473 0.43939653 0.33655212 0.49831399]


In [32]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [33]:
print(X_norm[0,:])
print(y)

[ 1.15563073 -0.49443795 -1.17934447  1.20371316 -0.55551984 -0.74051504
  0.89197269  0.80575756  0.46337918  0.50980485 -0.29183867 -0.4476351
  0.56969831 -1.68998664 -0.38244559  1.13253179]
[0 0 3 0 0 0 0 3 3 0 0 1 3 6 6 6 1 0 3 0 1 1 0 1 5 4 4 0 0 0 5 0 0 1 3 0 0
 1 3 5 5 1 5 1 0 0 6 0 0 0 0 5 4 6 0 0 1 1 1 1 3 3 2 0 0 0 0 0 0 0 0 1 6 3
 0 0 2 6 1 1 2 6 3 1 0 6 3 1 5 4 2 2 3 0 0 1 0 5 0 6 1]


In [34]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

### 1.3 Vectorización de regresión logística

Se utilizará múltiples modelos de regresión logística uno contra todos para construir un clasificador de clases múltiples. Dado que hay 10 clases, deberá entrenar 10 clasificadores de regresión logística separados. Para que esta capacitación sea eficiente, es importante asegurarse de que el código esté bien vectorizado.

En esta sección, se implementará una versión vectorizada de regresión logística que no emplea ningún bucle "for".

Para probar la regresión logística vectorizada, se usara datos personalizados como se definen a continuación.

<a id="section1"></a>
#### 1.3.1 Vectorización de la funcion de costo

Se inicia escribiendo una versión vectorizada de la función de costo. En la regresión logística (no regularizada), la función de costo es

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log \left( h_\theta\left( x^{(i)} \right) \right) - \left(1 - y^{(i)} \right) \log \left(1 - h_\theta \left( x^{(i)} \right) \right) \right] $$

Para calcular cada elemento en la suma, tenemos que calcular $h_\theta(x^{(i)})$ para cada ejemplo $i$, donde $h_\theta(x^{(i)}) = g(\theta^T x^{(i)})$ y $g(z) = \frac{1}{1+e^{-z}}$ es la funcion sigmoidea. Resulta que podemos calcular esto rápidamente para todos los ejemplos usando la multiplicación de matrices. Definamos $X$ y $\theta$ como

$$ X = \begin{bmatrix} - \left( x^{(1)} \right)^T - \\ - \left( x^{(2)} \right)^T - \\ \vdots \\ - \left( x^{(m)} \right)^T - \end{bmatrix} \qquad \text{and} \qquad \theta = \begin{bmatrix} \theta_0 \\ \theta_1 \\ \vdots \\ \theta_n \end{bmatrix} $$

Luego, de calcular el producto matricial $X\theta$, se tiene:

$$ X\theta = \begin{bmatrix} - \left( x^{(1)} \right)^T\theta - \\ - \left( x^{(2)} \right)^T\theta - \\ \vdots \\ - \left( x^{(m)} \right)^T\theta - \end{bmatrix} = \begin{bmatrix} - \theta^T x^{(1)}  - \\ - \theta^T x^{(2)} - \\ \vdots \\ - \theta^T x^{(m)}  - \end{bmatrix} $$

En la última igualdad, usamos el hecho de que $a^Tb = b^Ta$ if $a$ y $b$ son vectores. Esto permite calcular los productos $\theta^T x^{(i)}$ para todos los ejemplos $i$ en una linea de codigo.

#### 1.3.2 Vectorización del gradiente

Recordemos que el gradiente del costo de regresión logística (no regularizado) es un vector donde el elemento $j^{th}$ se define como
$$ \frac{\partial J }{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^m \left( \left( h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x_j^{(i)} \right) $$

Para vectorizar esta operación sobre el conjunto de datos, se inicia escribiendo todas las derivadas parciales explícitamente para todos $\theta_j$,

$$
\begin{align*}
\begin{bmatrix}
\frac{\partial J}{\partial \theta_0} \\
\frac{\partial J}{\partial \theta_1} \\
\frac{\partial J}{\partial \theta_2} \\
\vdots \\
\frac{\partial J}{\partial \theta_n}
\end{bmatrix} = &
\frac{1}{m} \begin{bmatrix}
\sum_{i=1}^m \left( \left(h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x_0^{(i)}\right) \\
\sum_{i=1}^m \left( \left(h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x_1^{(i)}\right) \\
\sum_{i=1}^m \left( \left(h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x_2^{(i)}\right) \\
\vdots \\
\sum_{i=1}^m \left( \left(h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x_n^{(i)}\right) \\
\end{bmatrix} \\
= & \frac{1}{m} \sum_{i=1}^m \left( \left(h_\theta\left(x^{(i)}\right) - y^{(i)} \right)x^{(i)}\right) \\
= & \frac{1}{m} X^T \left( h_\theta(x) - y\right)
\end{align*}
$$

donde

$$  h_\theta(x) - y =
\begin{bmatrix}
h_\theta\left(x^{(1)}\right) - y^{(1)} \\
h_\theta\left(x^{(2)}\right) - y^{(2)} \\
\vdots \\
h_\theta\left(x^{(m)}\right) - y^{(m)}
\end{bmatrix} $$

Nota $x^{(i)}$ es un vector, mientras $h_\theta\left(x^{(i)}\right) - y^{(i)}$ es un escalar(simple número).
Para comprender el último paso de la derivación, dejemos $\beta_i = (h_\theta\left(x^{(m)}\right) - y^{(m)})$ y
observar que:

$$ \sum_i \beta_ix^{(i)} = \begin{bmatrix}
| & | & & | \\
x^{(1)} & x^{(2)} & \cdots & x^{(m)} \\
| & | & & |
\end{bmatrix}
\begin{bmatrix}
\beta_1 \\
\beta_2 \\
\vdots \\
\beta_m
\end{bmatrix} = x^T \beta
$$

donde los valores $\beta_i = \left( h_\theta(x^{(i)} - y^{(i)} \right)$.

La expresión anterior nos permite calcular todas las derivadas parciales sin bucles.
Si se siente cómodo con el álgebra lineal, le recomendamos que trabaje con las multiplicaciones de matrices anteriores para convencerse de que la versión vectorizada hace los mismos cálculos.

<div class="alert alert-box alert-warning">
** Consejo de depuración: ** El código de vectorización a veces puede ser complicado. Una estrategia común para la depuración es imprimir los tamaños de las matrices con las que está trabajando usando la propiedad `shape` de las matrices` numpy`.

Por ejemplo, dada una matriz de datos $X$ de tamaño $100\veces 20$ (100 ejemplos, 20 características) y $\theta$, un vector con tamaño $20$, puede observar que `np.dot (X, theta) `es una operación de multiplicación válida, mientras que` np.dot (theta, X) `no lo es.

Además, si tiene una versión no vectorizada de su código, puede comparar la salida de su código vectorizado y el código no vectorizado para asegurarse de que produzcan las mismas salidas.</div>
<a id="lrCostFunction"></a>

In [35]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [36]:
def calcularCosto(theta, X, y):
    # Inicializar algunos valores utiles
    m = y.size  # numero de ejemplos de entrenamiento

    J = 0
    h = sigmoid(X.dot(theta.T))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    return J

In [37]:
def descensoGradiente(theta, X, y, alpha, num_iters):
    # Inicializa algunos valores
    m = y.shape[0] # numero de ejemplos de entrenamiento

    # realiza una copia de theta, el cual será acutalizada por el descenso por el gradiente
    theta = theta.copy()
    J_history = []

    for i in range(num_iters):
        h = sigmoid(X.dot(theta.T))
        theta = theta - (alpha / m) * (h - y).dot(X)

        J_history.append(calcularCosto(theta, X, y))
    return theta, J_history

In [38]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

#### 1.3.3 Vectorización regularizada de la regresión logística

Una vez implementada la vectorización para la regresión logística, corresponde agregarar regularización a la función de costo.
Para la regresión logística regularizada, la función de costo se define como

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log \left(h_\theta\left(x^{(i)} \right)\right) - \left( 1 - y^{(i)} \right) \log\left(1 - h_\theta \left(x^{(i)} \right) \right) \right] + \frac{\lambda}{2m} \sum_{j=1}^n \theta_j^2 $$

Tomar en cuenta que no debería regularizarse $\theta_0$ que se usa para el término de sesgo. En consecuencia, la derivada parcial del costo de regresión logística regularizado para $\theta_j$ se define como

$$
\begin{align*}
& \frac{\partial J(\theta)}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)}  & \text{for } j = 0 \\
& \frac{\partial J(\theta)}{\partial \theta_0} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_\theta\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m} \theta_j & \text{for } j  \ge 1
\end{align*}
$$

<div class="alert alert-box alert-warning">
** Python/numpy Consejo: ** Al implementar la vectorización para la regresión logística regularizada, a menudo es posible que solo desee sumar y actualizar ciertos elementos de $\theta$. En `numpy`, puede indexar en las matrices para acceder y actualizar solo ciertos elementos.

Por ejemplo, A [:, 3: 5] = B [:, 1: 3] reemplazará las columnas con índice 3 a 5 de A con las columnas con índice 1 a 3 de B.   
Para seleccionar columnas (o filas) hasta el final de la matriz, puede dejar el lado derecho de los dos puntos en blanco.
Por ejemplo, A [:, 2:] solo devolverá elementos desde $3^{rd}$ a las últimas columnas de $A$.Si deja el tamaño de la mano izquierda de los dos puntos en blanco, seleccionará los elementos del principio de la matriz.
Por ejemplo, A [:,: 2] selecciona las dos primeras columnas y es equivalente a A [:, 0: 2]. Además, puede utilizar índices negativos para indexar matrices desde el final.
Por lo tanto, A [:,: -1] selecciona todas las columnas de A excepto la última columna, y A [:, -5:] selecciona la columna $5^{th}$ desde el final hasta la última columna.

Por lo tanto, podría usar esto junto con las operaciones de suma y potencia ($^{**}$) para calcular la suma de solo los elementos que le interesan (por ejemplo, `np.sum (z[1:]**2)`).
</div>


1.4 Clasificacion One-vs-all

En esta parte del ejercicio, se implementará la clasificación de uno contra todos mediante el entrenamiento de múltiples clasificadores de regresión logística regularizados, uno para cada una de las clases  𝐾  en nuestro conjunto de datos. En el conjunto de datos de dígitos escritos a mano,  𝐾=10 , pero su código debería funcionar para cualquier valor de  𝐾 .

El argumento y de esta función es un vector de etiquetas de 0 a 9. Al entrenar el clasificador para la clase  𝑘∈{0,...,𝐾−1} , querrá un vector K-dimensional de etiquetas  𝑦 , donde  𝑦𝑗 𝑖𝑛0,1  indica si la instancia de entrenamiento  𝑗𝑡ℎ  pertenece a la clase  𝑘   (𝑦𝑗=1) , o si pertenece a una clase diferente  (𝑦𝑗=0) .

Además, se utiliza optimize.minimize de scipy para este ejercicio.

In [39]:
# 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Conjunto de entrenamiento:", X_train.shape)
print("Conjunto de prueba:", X_test.shape)

Conjunto de entrenamiento: (80, 16)
Conjunto de prueba: (21, 16)


In [40]:
num_labels = 7
lambda_ = 0.1

all_theta = OneVsAllOM(X_train, y_train, num_labels, lambda_)

In [41]:
# Precisión en entrenamiento
y_pred_train = predictOneVsAll(all_theta, X_train)
print("Precisión en entrenamiento: {:.2f}%".format(np.mean(y_pred_train == y_train) * 100))

# Precisión en prueba
y_pred_test = predictOneVsAll(all_theta, X_test)
print("Precisión en prueba: {:.2f}%".format(np.mean(y_pred_test == y_test) * 100))

Precisión en entrenamiento: 100.00%
Precisión en prueba: 100.00%


In [42]:

def OneVsAll(X, y, num_labels, lambda_=0):
    alpha = 0.01
    num_iters = 3000  # menos iteraciones, más práctico en zoo (101 ejemplos)

    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar columna de unos a X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in range(num_labels):
        initial_theta = np.zeros(n + 1)

        # Etiquetas binarias: 1 si clase == c, 0 en caso contrario
        y_actual = np.where(y == c, 1, 0)

        theta, J_history = descensoGradiente(initial_theta, X, y_actual,
                                             alpha, num_iters, lambda_)

        all_theta[c] = theta

        # Opcional: graficar convergencia
        plt.plot(np.arange(len(J_history)), J_history, lw=2, label=f"Clase {c}")

    plt.xlabel('Iteraciones')
    plt.ylabel('Costo J')
    plt.legend()
    plt.show()

    return all_theta


In [43]:
import numpy as np
from scipy import optimize

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def lrCostFunction(theta, X, y, lambda_):
    """
    Función de costo regularizada para regresión logística.
    """
    m = len(y)
    h = sigmoid(X.dot(theta))

    # costo
    J = (-1/m) * (y.T.dot(np.log(h)) + (1-y).T.dot(np.log(1-h)))
    J += (lambda_/(2*m)) * np.sum(np.square(theta[1:]))

    # gradiente
    grad = (1/m) * (X.T.dot(h - y))
    grad[1:] += (lambda_/m) * theta[1:]

    return J, grad


def OneVsAllOM(X, y, num_labels, lambda_):
    """
    Entrena num_labels clasificadores logísticos y retorna
    la matriz all_theta con cada clasificador en una fila.
    """
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar columna de unos (bias)
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}

        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                args=(X, (y == c).astype(int), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [44]:
# Regularización
lambda_ = 0.1

# Entrenar One-vs-All con optimización
all_theta = OneVsAllOM(X, y, num_labels, lambda_)

# Mostrar dimensiones de la matriz de parámetros
print(all_theta.shape)

(7, 17)


In [45]:
print(all_theta)

[[-1.48436660e+00  1.51562269e+00 -5.36634974e-01 -1.52249261e+00
   2.29657787e+00 -2.53161267e-01  5.73916883e-02  7.69750275e-02
   4.23983899e-01  4.71943834e-01  7.73444748e-01 -4.75189016e-01
   2.46812659e-02  4.43840337e-02  2.91057337e-01  1.49260772e-01
   7.20096306e-01]
 [-4.86562173e+00 -4.92065596e-01  2.81067076e+00  3.40172676e-01
  -4.40272600e-01  7.56529991e-01 -8.78863783e-03  1.34124028e-01
  -1.06861978e+00  6.50029751e-01  4.37792778e-01 -2.68475843e-01
  -4.12862127e-01 -4.96119117e-01  6.17191396e-01  1.12915629e-01
   9.43154998e-02]
 [-6.58180424e+00 -1.76911474e+00 -2.39922526e+00  2.47820279e-01
  -1.78862507e+00 -8.28311503e-01 -1.96355073e+00 -2.10562183e-01
   2.17629932e-01  1.84141365e+00 -4.80069579e-01  6.07887144e-01
  -2.19462849e+00 -7.18937694e-01  1.51520883e+00 -3.86257520e-01
  -2.86452955e-02]
 [-7.37730182e+00 -1.93229170e-01 -2.52579367e-01  1.33667309e+00
  -9.00193449e-01 -1.84135012e-01  4.21861179e-01 -2.43796784e-01
   6.59806392e-01  

1.4.1 Prediccion One-vs-all

Después de entrenar el clasificador de one-vs-all, se puede usarlo para predecir el dígito contenido en una imagen determinada. Para cada entrada, debe calcular la "probabilidad" de que pertenezca a cada clase utilizando los clasificadores de regresión logística entrenados. La función de predicción one-vs-all seleccionará la clase para la cual el clasificador de regresión logística correspondiente genera la probabilidad más alta y devolverá la etiqueta de clase (0, 1, ..., K-1) como la predicción para el ejemplo de entrada.

In [46]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    - all_theta: matriz (K x n+1) con los parámetros entrenados.
    - X: matriz (m x n) con las características (sin el bias).
    """
    m = X.shape[0]
    num_labels = all_theta.shape[0]

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # Probabilidades para cada clase
    probs = sigmoid(X.dot(all_theta.T))

    # Predicción = índice de la probabilidad mayor
    p = np.argmax(probs, axis=1)

    return p

Una vez que haya terminado, se llama a la función predictOneVsAll usando el valor aprendido de  𝜃 . Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 95,1% (es decir, clasifica correctamente el 95,1% de los ejemplos del conjunto de entrenamiento).

In [48]:
print(X.shape)  # (101, 16)

pred = predictOneVsAll(all_theta, X)
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))

# Tomamos el último animal como prueba
XPrueba = X[100:101, :].copy()
print(XPrueba.shape)  # (1, 16)

# Agregar bias
XPrueba = np.concatenate([np.ones((XPrueba.shape[0], 1)), XPrueba], axis=1)
print(XPrueba.shape)  # (1, 17)

# Predicción
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)
print("Predicción:", p)

# Clase real
print("Etiqueta real:", y[100:101])

(101, 16)
Precisión del conjunto de entrenamiento: 100.00%
(1, 16)
(1, 17)
Predicción: [1]
Etiqueta real: [1]
